In [ ]:
print("GPU Check")
!head -n 3 /proc/meminfo
!nvidia-smi -L
!nvidia-smi

In [ ]:
# from google.colab import drive
# drive.flush_and_unmount()
# !rm -rf /content/drive
# drive.mount('/content/drive')
# print('Done!')

In [ ]:
print("Python install")
!pip install pytorch_lightning
!pip install torchcrepe==0.0.17;
!pip install praat-parselmouth==0.4.1;
!pip install scikit-image==0.19.3;
!pip install ipython==8.5.0;
!pip install ipykernel==6.16.2;
!pip install pyloudnorm==0.1.0;
!pip install webrtcvad==2.0.10;
!pip install h5py==3.7.0;
!pip install einops==0.5.0;
!pip install pycwt==0.3.0a22;
!pip install torchmetrics==0.5;
!pip install pytorch_lightning==1.3.3;
!pip install pyworld;
!pip uninstall numba --yes;
!pip install numba;
!pip install pydub

In [ ]:
import os
from IPython.display import clear_output

print("Diff-SVC install")

agree = True

if agree:
	!rm -rf sample_data
	!python -m pip install --upgrade pip wheel --quiet
	!pip uninstall gdown -y --quiet
	#!pip install git+https://github.com/justinjohn0306/gdown.git --quiet
	!pip install pydub fuzzywuzzy python-Levenshtein pyworld==0.3.1 --quiet

	#install aria2
	!sudo apt-get install aria2  &> /dev/null
	!apt install wget curl ca-certificates &> /dev/null
	!wget -N git.io/aria2.sh &> /dev/null && chmod +x aria2.sh &> /dev/null
	!echo 1|./aria2.sh &> /dev/null
	!echo 12|./aria2.sh &> /dev/null
	!echo 6|./aria2.sh &> /dev/null
	!pip install --pre torchtext==0.6.0 --no-deps --quiet

	fork = "UtaUtaUtau's Repo"
	sample_rate = '24kHz'

	print('Installing Diff-SVC')
	if fork == "Official Diff-SVC Repo":
		!git clone -q https://github.com/prophesier/diff-svc
	else:
		!git clone -q --branch harvest-preprocess https://github.com/UtaUtaUtau/diff-svc

	%cd "diff-svc"
	!pip install -r requirements_short.txt --quiet
	!pip install tensorboard<2.9,>=2.8 --quiet
	#!pip install --upgrade numpy==1.23.0 scipy==1.9.3 --quiet
	%reload_ext tensorboard

	%mkdir -p checkpoints

	hifigan_24k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/hifigan_24k.zip"
	hifigan_44k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/hifigan_44k.zip"
	checkpoints = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/checkpoints.zip"

	!aria2c --file-allocation=none -c -x 10 -s 10 {checkpoints} -q
	!unzip checkpoints.zip
	!rm checkpoints.zip

	if sample_rate == "44.1kHz":
		!aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_44k} -q
		!unzip hifigan_44k.zip -d checkpoints
		!rm hifigan_44k.zip
		config_path = "training/config_nsf.yaml"
		slay = "44100"
	else:
		!aria2c --file-allocation=none -c -x 10 -s 10 {hifigan_24k} -q
		!unzip hifigan_24k.zip
		!rm hifigan_24k.zip
		config_path = "training/config.yaml"
		!rm {config_path}
		!wget "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/config.yaml" -O {config_path} -q
		slay = "24000"
	clear_output()
	print('Done!')

else:
	print("So you did not agree...")

In [ ]:
print("Path setting")
singer_name = 'test_kjh_diff' #@param {type: "string"}
input_filepath = '/content/drive/MyDrive/kjh.wav' #@param {type: "string"}


In [ ]:
import re

%cd "/content/diff-svc/"

os.environ['PYTHONPATH']='.'

!CUDA_VISIBLE_DEVICES=0

from utils.hparams import hparams
from preprocessing.data_gen_utils import get_pitch_parselmouth,get_pitch_crepe
import os
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import utils
import librosa
import torchcrepe
from infer import *
import logging
from infer_tools.infer_tool import *

print("Get Model")

logging.getLogger('numba').setLevel(logging.WARNING)

project_name = singer_name
# 모델 파일이 저장된 디렉토리 경로
model_dir = f'/content/drive/MyDrive/{singer_name}/checkpoint/'

# 모든 파일 이름을 가져옵니다.
all_files = os.listdir(model_dir)

# 'model_ckpt_steps_x.ckpt' 형식의 파일만 걸러냅니다.
model_files = [f for f in all_files if re.match(r'model_ckpt_steps_\d+\.ckpt', f)]

# 파일 이름에서 숫자 부분만 추출하고 이를 정수로 변환합니다.
model_nums = [int(re.findall(r'\d+', f)[0]) for f in model_files]

# 가장 큰 숫자를 찾습니다.
latest_model_num = max(model_nums)

# 가장 큰 숫자를 가진 모델 파일의 이름을 만듭니다.
model_path = os.path.join(model_dir, f'model_ckpt_steps_{latest_model_num}.ckpt')
print(model_path)
config_path=f'/content/drive/MyDrive/{singer_name}/checkpoint/config.yaml'
hubert_gpu=True
svc_model = Svc(project_name,config_path,hubert_gpu, model_path)
print('model loaded')

In [ ]:
import os

print("Output")

wav_in_folder = "/content/drive/MyDrive/test_kjh_diff" #@param {type: "string"}

# 모든 wav 파일 가져오기
wav_in_files = [os.path.join(wav_in_folder, file) for file in os.listdir(wav_in_folder) if file.endswith('.wav')]

# 출력 경로 설정
output_folder = f"/content/drive/MyDrive/{singer_name}/result"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # 경로가 존재하지 않는 경우 폴더 생성
key = 0


pndm_speedup = 100

add_noise_step = 1000

thre = 0.05
use_crepe= True
use_pe=False
use_gt_mel= True
# 각 입력 파일에 대해 처리
# 각 입력 파일에 대해 처리
for wav_in in wav_in_files:
    # 출력 파일 이름 생성 (입력 파일 이름에 프로젝트 이름 추가)
    wav_out_base = os.path.basename(wav_in)  # 입력 파일의 기본 이름 (확장자 포함)
    wav_out_base_no_ext = os.path.splitext(wav_out_base)[0]  # 확장자 제거
    wav_out = f"{wav_out_base_no_ext}_{singer_name}.wav"  # 프로젝트 이름 추가

    # 중복된 파일 이름이 존재할 경우 '_re' 추가
    while os.path.exists(f"{output_folder}/{wav_out}"):
        wav_out_base_no_ext = wav_out_base_no_ext + '_re'
        wav_out = f"{wav_out_base_no_ext}_{singer_name}.wav"

    # 모델 실행 및 결과 저장
    f0_tst, f0_pred, audio = run_clip(svc_model, file_path=wav_in, key=key, acc=pndm_speedup, use_crepe=use_crepe, use_pe=use_pe, thre=thre, use_gt_mel=use_gt_mel, add_noise_step=add_noise_step, project_name=project_name, out_path=f"{output_folder}/{wav_out}")
